In [1]:
!pip install nba_api

In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.patches import Circle, Rectangle, Arc, ConnectionPatch, Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import LinearSegmentedColormap, ListedColormap, BoundaryNorm
from matplotlib.path import Path
from matplotlib.patches import PathPatch
import seaborn as sns
from scipy.stats import norm, gaussian_kde, percentileofscore
from itertools import chain
import pandas as pd
import numpy as np
from datetime import datetime
from nba_api.stats.static import players
from nba_api.stats.endpoints import leaguedashplayerstats, commonplayerinfo, playercareerstats, shotchartdetail
import pickle
from copy import deepcopy
import dash_bootstrap_components as dbc

In [3]:
# list of all active players
all_players = players.get_active_players()

# stats dataframes
adv_stats = leaguedashplayerstats.LeagueDashPlayerStats(season="2022-23", measure_type_detailed_defense='Advanced').get_data_frames()[0]
base_stats = leaguedashplayerstats.LeagueDashPlayerStats(season="2022-23", measure_type_detailed_defense='Base').get_data_frames()[0]

# extract only necessary values and concatenate
df = adv_stats[['PLAYER_ID', 'PLAYER_NAME', 'USG_PCT', 'AST_PCT', 'OREB_PCT', 'TS_PCT', 'PACE_PER40', 'DREB_PCT', 'E_DEF_RATING']].copy()
df2 = base_stats[['BLK', 'STL']].copy()
pos_df = pd.concat([df, df2], axis='columns')

# adding new columns to the df to store percentile scores for all chosen stats
pos_df['USG_POS'] = pos_df.apply(lambda x: (percentileofscore(df['USG_PCT'], x['USG_PCT']))/10, axis=1)
pos_df['AST_POS'] = pos_df.apply(lambda x: (percentileofscore(df['AST_PCT'], x['AST_PCT']))/10, axis=1)
pos_df['OREB_POS'] = pos_df.apply(lambda x: (percentileofscore(df['OREB_PCT'], x['OREB_PCT']))/10, axis=1)
pos_df['TS_POS'] = pos_df.apply(lambda x: (percentileofscore(df['TS_PCT'], x['TS_PCT']))/10, axis=1)
pos_df['PP40_POS'] = pos_df.apply(lambda x: (percentileofscore(df['PACE_PER40'], x['PACE_PER40']))/10, axis=1)
pos_df['DREB_POS'] = pos_df.apply(lambda x: (percentileofscore(df['DREB_PCT'], x['DREB_PCT']))/10, axis=1)
pos_df['E_DEF_POS'] = pos_df.apply(lambda x: (percentileofscore(df['E_DEF_RATING'], x['E_DEF_RATING']))/10, axis=1)
pos_df['BLK_POS'] = pos_df.apply(lambda x: (percentileofscore(df2['BLK'], x['BLK']))/10, axis=1)
pos_df['STL_POS'] = pos_df.apply(lambda x: (percentileofscore(df2['STL'], x['STL']))/10, axis=1)

# drop old columns
pos_df.drop(pos_df.columns[[2,3,4,5,6,7,8,9,10]], axis=1, inplace=True)

# add total composite offense + defense scores
pos_df['OFF_SCORE'] = pos_df.apply(lambda x: sum(x[2:7]), axis=1)
pos_df['DEF_SCORE'] = pos_df.apply(lambda x: sum(x[7:11]), axis=1)

# hexbin league df
grouped_shots_df = pd.read_csv('league_shots_by_dist.csv')
with open('srcdata/league_hexbin_stats.pickle', 'rb') as f:
    league_hexbin_stats = pickle.load(f)

In [4]:
def draw_plotly_court(fig, fig_width=600, margins=10):

    import numpy as np
        
    # From: https://community.plot.ly/t/arc-shape-with-path/7205/5
    def ellipse_arc(x_center=0.0, y_center=0.0, a=10.5, b=10.5, start_angle=0.0, end_angle=2 * np.pi, N=200, closed=False):
        t = np.linspace(start_angle, end_angle, N)
        x = x_center + a * np.cos(t)
        y = y_center + b * np.sin(t)
        path = f'M {x[0]}, {y[0]}'
        for k in range(1, len(t)):
            path += f'L{x[k]}, {y[k]}'
        if closed:
            path += ' Z'
        return path

    fig_height = fig_width * (470 + 2 * margins) / (500 + 2 * margins)
    fig.update_layout(width=fig_width, height=fig_height)

    # Set axes ranges
    fig.update_xaxes(range=[-250 - margins, 250 + margins])
    fig.update_yaxes(range=[-52.5 - margins, 417.5 + margins])

    threept_break_y = 89.47765084
    three_line_col = "#777777"
    main_line_col = "#777777"

    fig.update_layout(
        # Line Horizontal
        margin=dict(l=20, r=20, t=20, b=20),
        paper_bgcolor="white",
        plot_bgcolor="white",
        yaxis=dict(
            scaleanchor="x",
            scaleratio=1,
            showgrid=False,
            zeroline=False,
            showline=False,
            ticks='',
            showticklabels=False,
            fixedrange=True,
        ),
        xaxis=dict(
            showgrid=False,
            zeroline=False,
            showline=False,
            ticks='',
            showticklabels=False,
            fixedrange=True,
        ),
        shapes=[
            dict(
                type="rect", x0=-250, y0=-52.5, x1=250, y1=417.5,
                line=dict(color=main_line_col, width=1),
                # fillcolor='#333333',
                layer='below'
            ),
            dict(
                type="rect", x0=-80, y0=-52.5, x1=80, y1=137.5,
                line=dict(color=main_line_col, width=1),
                # fillcolor='#333333',
                layer='below'
            ),
            dict(
                type="rect", x0=-60, y0=-52.5, x1=60, y1=137.5,
                line=dict(color=main_line_col, width=1),
                # fillcolor='#333333',
                layer='below'
            ),
            dict(
                type="circle", x0=-60, y0=77.5, x1=60, y1=197.5, xref="x", yref="y",
                line=dict(color=main_line_col, width=1),
                # fillcolor='#dddddd',
                layer='below'
            ),
            dict(
                type="line", x0=-60, y0=137.5, x1=60, y1=137.5,
                line=dict(color=main_line_col, width=1),
                layer='below'
            ),

            dict(
                type="rect", x0=-2, y0=-7.25, x1=2, y1=-12.5,
                line=dict(color="#ec7607", width=1),
                fillcolor='#ec7607',
            ),
            dict(
                type="circle", x0=-7.5, y0=-7.5, x1=7.5, y1=7.5, xref="x", yref="y",
                line=dict(color="#ec7607", width=1),
            ),
            dict(
                type="line", x0=-30, y0=-12.5, x1=30, y1=-12.5,
                line=dict(color="#ec7607", width=1),
            ),

            dict(type="path",
                 path=ellipse_arc(a=40, b=40, start_angle=0, end_angle=np.pi),
                 line=dict(color=main_line_col, width=1), layer='below'),
            dict(type="path",
                 path=ellipse_arc(a=237.5, b=237.5, start_angle=0.386283101, end_angle=np.pi - 0.386283101),
                 line=dict(color=main_line_col, width=1), layer='below'),
            dict(
                type="line", x0=-220, y0=-52.5, x1=-220, y1=threept_break_y,
                line=dict(color=three_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-220, y0=-52.5, x1=-220, y1=threept_break_y,
                line=dict(color=three_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=220, y0=-52.5, x1=220, y1=threept_break_y,
                line=dict(color=three_line_col, width=1), layer='below'
            ),

            dict(
                type="line", x0=-250, y0=227.5, x1=-220, y1=227.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=250, y0=227.5, x1=220, y1=227.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-90, y0=17.5, x1=-80, y1=17.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-90, y0=27.5, x1=-80, y1=27.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-90, y0=57.5, x1=-80, y1=57.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-90, y0=87.5, x1=-80, y1=87.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=90, y0=17.5, x1=80, y1=17.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=90, y0=27.5, x1=80, y1=27.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=90, y0=57.5, x1=80, y1=57.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=90, y0=87.5, x1=80, y1=87.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),

            dict(type="path",
                 path=ellipse_arc(y_center=417.5, a=60, b=60, start_angle=-0, end_angle=-np.pi),
                 line=dict(color=main_line_col, width=1), layer='below'),

        ]
    )
    return True

In [5]:
def filt_hexbins(hexbin_stats, min_threshold=0.0):

    from copy import deepcopy

    filt_hexbin_stats = deepcopy(hexbin_stats)
    temp_len = len(filt_hexbin_stats['freq_by_hex'])
    filt_array = [i > min_threshold for i in filt_hexbin_stats['freq_by_hex']]
    for k, v in filt_hexbin_stats.items():
        if type(v) != int:
            if len(v) == temp_len:
                filt_hexbin_stats[k] = [v[i] for i in range(temp_len) if filt_array[i]]

    return filt_hexbin_stats

In [6]:
def calculate_percentiles(season_data, stat):
    """Calculate the percentiles for a given stat across all players."""
    return np.percentile(season_data[stat], [25, 80])

def calculate_percentiles2(season_data, stat):
    """Calculate the 50th and 75th percentiles for a given stat across all players."""
    return np.percentile(season_data[stat], [50, 75])

# Now you can calculate USG% and Assist% percentiles for the season
usg_percentiles = calculate_percentiles(adv_stats, 'USG_PCT')
assist_percentiles = calculate_percentiles(adv_stats, 'AST_PCT')

def determine_badge_color(player_stat, percentiles):
    """Determine badge color based on player's stat and percentiles."""
    if player_stat >= percentiles[1]:  # Above 75th percentile, return green
        return {'font-size': '20px', 'font-family': 'verdana', 'background-color':'#2E8B57'}
    elif player_stat <= percentiles[0]:  # Below 25th percentile, return red
        return {'font-size': '20px', 'font-family': 'verdana', 'background-color':'#8B0000'}
    else:
        # Calculate gradient here (can be more complex based on your requirements)
        return {'font-size': '20px', 'font-family': 'verdana', 'background-color':'#A9A9A9'}

def determine_catch_shoot_badge_color(usg_stat, usg_percentiles, ts_stat, ts_percentiles):
    """Determine badge color for catch and shoot efficiency based on usage and true shooting percentiles."""
    if usg_stat <= usg_percentiles[0] and ts_stat >= ts_percentiles[1]:  # Bottom half in USG% and top 25% in TS%
        return {'font-size': '20px', 'font-family': 'verdana', 'background-color':'#2E8B57'}
    else:
        return {'font-size': '20px', 'font-family': 'verdana', 'background-color':'#A9A9A9'}

def evaluate_creator_facilitator_category(usg_color, ast_color):
    if {'font-size': '20px', 'font-family': 'verdana', 'background-color':'#2E8B57'} in [usg_color, ast_color]:
        return {'font-size': '20px', 'font-family': 'verdana', 'background-color':'#2E8B57'}
    elif {'font-size': '20px', 'font-family': 'verdana', 'background-color':'#8B0000'} in [usg_color, ast_color]:
        return {'font-size': '20px', 'font-family': 'verdana', 'background-color':'#8B0000'}
    else:
        return {'font-size': '20px', 'font-family': 'verdana', 'background-color':'#A9A9A9'}

In [7]:
fig2 = px.parallel_coordinates(pos_df, dimensions=['USG_POS', 'AST_POS', 'OREB_POS', 'TS_POS', 'PP40_POS', 'DREB_POS', 'E_DEF_POS', 'BLK_POS', 'STL_POS'],
                                  labels={'USG_POS':'Usage %', 'AST_POS':'Assists', 'OREB_POS':'Offensive Rebound %', 'TS_POS':'True Shooting %', 'PP40_POS':'Pace Factor', 'DREB_POS':'Defensive Rebound %', 'E_DEF_POS':'Defensive Rating', 'BLK_POS':'Blocks', 'STL_POS':'Steals'},
                                  width=1000, height=500)
fig2.update_layout(clickmode='event+select')
fig2.update_layout(hovermode='closest')

# Initialize the Dash app
app = dash.Dash(__name__)

# tab 1: basic player info, offense/defense attributes, shot chart
tab1_layout = html.Div([
    # dropdown for player selection
    dcc.Dropdown(
        id='player-dropdown',
        options=[{'label': player['full_name'], 'value': player['id']} for player in all_players], placeholder="Select a player",
    ),
    # Display player information
    html.H4('BASIC INFO', style={'font-size': '15px', 'font-family': 'verdana'}),
    html.Div(id='player-info'), 

    html.Div(className='row', children=[
        html.Div([
            dcc.Markdown("""
                **Attributes**

                _Facilitator_
                + USG%: percentage of a team's offensive possessions used by an individual player
                + AST%: percentage of a team's possessions a player assists on while on the floor

                _Second Chance Creator_
                + OREB: offensive rebounds
                
                _Catch & Shoot_
                + 3P%: percentage of made 3 pointers
                + USG%
                
                _Movement Player_
                + PACE PER 40:  number of possessions per 40 minutes

                _Glass Cleaner_
                + DREB_PCT: ratio between the team's defensive rebounds and the same amount plus the opponent's offensive rebounds
                
                _Shot Blocker_
                + Blocks

                _On-Ball Defender_
                + E_DEF_RATING: player's efficiency at preventing the other team from scoring points
                
                _Pickpocket_
                + Steals

            """),
            html.Pre(id='attributes')
        ], className='two columns'),
    ]),

    
    # Button grids for offense and defense
    html.Div([
        # Offense grid
        html.Div([
            html.H2("OFFENSE", style={'font-size': '25px', 'font-family': 'verdana'}),
            dbc.Button("Facilitator", id='facilitator-button', style={'font-size': '20px', 'font-family': 'verdana'}),
            dbc.Button("Second Chance Creator", id='chance-creator-button', style={'font-size': '20px', 'font-family': 'verdana'}),
            dbc.Button("Movement Player", id='movement-button', style={'font-size': '20px', 'font-family': 'verdana'}),
            dbc.Button("Catch & Shoot", id='catch-shoot-button', style={'font-size': '20px', 'font-family': 'verdana'}),
        ], style={'display': 'inline-block', 'margin-right': '50px'}),

        # Defense grid
        html.Div([
            html.H2("DEFENSE", style={'font-size': '25px', 'font-family': 'verdana'}),
            dbc.Button("Glass Cleaner", id='glass-cleaner-button', style={'font-size': '20px', 'font-family': 'verdana'}),
            dbc.Button("On-Ball Defender", id='defender-button', style={'font-size': '20px', 'font-family': 'verdana'}),
            dbc.Button("Shot Blocker", id='shot-blocker-button', style={'font-size': '20px', 'font-family': 'verdana'}),
            dbc.Button("Pickpocket", id='pickpocket-button', style={'font-size': '20px', 'font-family': 'verdana'}),
        ], style={'display': 'inline-block'}),
    ]),

     # explain stat calculations
    html.Div(className='row', children=[
        html.Div([
            dcc.Markdown("""

            
                **Shot Chart**

                Displays the team's shot chart, with hover data and a color map denoting their performance in comparison to the league average.
            """),
            html.Pre(id='chart-blurb'),
        ], className='two columns')
    ]),

    # Display player shot chart
    dcc.Graph(id='shot-chart')
])

# tab 2: offense/defense spider charts (overlaying 2 players), display each player's total composite score
tab2_layout = html.Div([
    # explain spider charts
    html.Div(className='row', children=[
        html.Div([
            dcc.Markdown("""
                **Spider Charts**

                Select two players from the 2022-23 season and generate two spider charts 
                (one for offense and another for defense) directly comparing their stats."""),
            html.Pre(id='chart-blurb'),
        ], className='two columns')
    ]),
    
    # first dropdown
    dcc.Dropdown(
        id='dropdown1',
        options=[{'label': player['full_name'], 'value': player['id']} for player in all_players], placeholder="Select a player",
    ),
    # second dropdown
    dcc.Dropdown(
        id='dropdown2',
        options=[{'label': player['full_name'], 'value': player['id']} for player in all_players], placeholder="Select a player",
    ),
    
    dcc.Graph(id='spider-offense'),
    dcc.Graph(id='spider-defense'),
    dcc.Store(id='intermediate-fig'),
])

# tab 3: interactive filtering graph to narrow down player with specific stats a coach wants
tab3_layout = html.Div([
    # explain parallel coordinate chart
    html.Div(className='row', children=[
        html.Div([
            dcc.Markdown("""
                **Parallel Coordinate Plot**

                Select ranges along the vertical axes with your mouse in order to filter out players that match your desired skillset.
                """),
            html.Pre(id='chart-blurb'),
        ], className='two columns')
    ]),
    
    dcc.Graph(id='par-coord', figure=fig2),
])
        
# Define the layout of the app with tabs
app.layout = html.Div([
    dcc.Tabs(id='tabs', value='tab1', children=[
        dcc.Tab(label='Statistical Overview', value='tab1'),
        dcc.Tab(label='Performance Comparison', value='tab2'),
        dcc.Tab(label='Search & Filter', value='tab3'),
    ]),
    html.Div(id='tabs-content')
])

# Callback to update tab content based on user input
@app.callback(
    Output('tabs-content', 'children'),
    Input('tabs', 'value')
)
def switch_tabs(selected_tab):
    if selected_tab == 'tab1':
        return tab1_layout
    elif selected_tab == 'tab2':
        return tab2_layout
    elif selected_tab == 'tab3':
        return tab3_layout

# ALL Tab 1 callbacks:
# update player information, shot chart, and button colors based on dropdown selection
@app.callback(
    [Output('player-info', 'children'),
     Output('shot-chart', 'figure')],
    [Input('player-dropdown', 'value')]
)
def update_player(selected_player_id):
    if selected_player_id is None:
        return "", {}
        
    # Get player information
    player_info = commonplayerinfo.CommonPlayerInfo(player_id=selected_player_id)
    player_info_data = player_info.get_normalized_dict()
    
    position = player_info_data['CommonPlayerInfo'][0]['POSITION']
    height = player_info_data['CommonPlayerInfo'][0]['HEIGHT']
    weight = player_info_data['CommonPlayerInfo'][0]['WEIGHT']
    birthdate = datetime.strptime(player_info_data['CommonPlayerInfo'][0]['BIRTHDATE'], "%Y-%m-%dT%H:%M:%S")
    age = datetime.now().year - birthdate.year - ((datetime.now().month, datetime.now().day) < (birthdate.month, birthdate.day))

    # Format player information
    player_info_text = f"Age: {age}, Position: {position}, Height: {height}, Weight: {weight}"

    # create shot chart
    teamname = player_info_data['CommonPlayerInfo'][0]['TEAM_ABBREVIATION']
    with open('srcdata/' + teamname.lower() + '_hexbin_stats.pickle', 'rb') as f:
        team_hexbin_stats = pickle.load(f)

    max_freq = 0.002

    rel_hexbin_stats = deepcopy(team_hexbin_stats)
    base_hexbin_stats = deepcopy(league_hexbin_stats)
    rel_hexbin_stats['accs_by_hex'] = rel_hexbin_stats['accs_by_hex'] - base_hexbin_stats['accs_by_hex']
    rel_hexbin_stats = filt_hexbins(rel_hexbin_stats, min_threshold=0.0)

    xlocs = rel_hexbin_stats['xlocs']
    ylocs = rel_hexbin_stats['ylocs']
    accs_by_hex = rel_hexbin_stats['accs_by_hex']
    freq_by_hex = np.array([min(max_freq, i) for i in rel_hexbin_stats['freq_by_hex']])

    colorscale = 'RdYlBu_r'
    marker_cmin = -0.05
    marker_cmax = 0.05
    title_txt = teamname + ":<BR>Shot chart, <BR>(vs NBA average)"

    hexbin_text = [
        '<i>Accuracy: </i>' + str(round(accs_by_hex[i]*100, 1)) + '% (vs league avg)<BR>'
        '<i>Frequency: </i>' + str(round(freq_by_hex[i]*100, 2)) + '%'
        for i in range(len(freq_by_hex))
    ]
    ticktexts = ["Worse", "Average", "Better"]
    # logo_url = "https://d2p3bygnnzw9w3.cloudfront.net/req/202001161/tlogo/bbr/" + teamname + ".png"

    fig = go.Figure()
    draw_plotly_court(fig, fig_width=1000)
    fig.add_trace(go.Scatter(
        x=xlocs, y=ylocs, mode='markers', name='markers',
        text=hexbin_text,
        marker=dict(
            size=freq_by_hex, sizemode='area', sizeref=2. * max(freq_by_hex) / (11. ** 2), sizemin=2.5,
            color=accs_by_hex, colorscale=colorscale,
            colorbar=dict(
                thickness=15,
                x=0.84,
                y=0.87,
                yanchor='middle',
                len=0.2,
                title=dict(
                    text="<B>Accuracy</B>",
                    font=dict(
                        size=11,
                        color='#4d4d4d'
                    ),
                ),
                tickvals=[marker_cmin, (marker_cmin + marker_cmax) / 2, marker_cmax],
                ticktext=ticktexts,
                tickfont=dict(
                    size=11,
                    color='#4d4d4d'
                )
            ),
            cmin=marker_cmin, cmax=marker_cmax,
            line=dict(width=1, color='#333333'), symbol='hexagon',
        ),
        hoverinfo='text'
    ))

    fig.update_layout(
        title=dict(
            text=title_txt,
            y=0.9,
            x=0.19,
            xanchor='left',
            yanchor='middle'),
        font=dict(
            family="Arial, Tahoma, Helvetica",
            size=10,
            color="#7f7f7f"
            ),
    )
    
    return player_info_text, fig

@app.callback(
    [Output('facilitator-button', 'style'),
     Output('chance-creator-button', 'style'),
     Output('movement-button', 'style'),
     Output('catch-shoot-button', 'style'),
     Output('glass-cleaner-button', 'style'),
     Output('defender-button', 'style'),
     Output('shot-blocker-button', 'style'),
     Output('pickpocket-button', 'style')],
    [Input('player-dropdown', 'value')]
)
def update_buttons(selected_player_id):
    # Get player information
    basic_player_data = base_stats[base_stats['PLAYER_ID'] == selected_player_id].iloc[0]
    advanced_player_data = adv_stats[adv_stats['PLAYER_ID'] == selected_player_id].iloc[0]

    # calculate percentiles
    usg_percentiles = calculate_percentiles(adv_stats, 'USG_PCT')
    assist_percentiles = calculate_percentiles(adv_stats, 'AST_PCT')
    off_reb_percentiles = calculate_percentiles(base_stats, 'OREB')
    true_shooting_percentile = calculate_percentiles2(adv_stats, 'TS_PCT')
    usg_percentiles = calculate_percentiles2(adv_stats, 'USG_PCT')
    def_reb_percentiles = calculate_percentiles(adv_stats, 'DREB_PCT')
    shot_block_percentiles = calculate_percentiles(base_stats, 'BLK')
    steals_percentiles = calculate_percentiles(base_stats, 'STL')
    hustle_percentiles = calculate_percentiles(adv_stats, 'E_DEF_RATING')
    pace_percentiles = calculate_percentiles(adv_stats, 'PACE_PER40')

    # calculate percentile cutoffs for badge colors
    usg_badge_color = determine_badge_color(advanced_player_data['USG_PCT'], usg_percentiles)
    assist_badge_color = determine_badge_color(advanced_player_data['AST_PCT'], assist_percentiles)
    off_reb_badge_color = determine_badge_color(basic_player_data['OREB'], off_reb_percentiles)
    def_reb_badge_color = determine_badge_color(advanced_player_data['DREB_PCT'], def_reb_percentiles)
    shot_block_badge_color = determine_badge_color(basic_player_data['BLK'], shot_block_percentiles)
    steal_badge_color = determine_badge_color(basic_player_data['STL'], steals_percentiles)
    hustle_badge_color = determine_badge_color(advanced_player_data['E_DEF_RATING'], hustle_percentiles)
    pace_badge_color = determine_badge_color(advanced_player_data['PACE_PER40'], pace_percentiles)

    # Calculate combined badge color for Facilitator/Creator
    facilitator_creator_badge_color = evaluate_creator_facilitator_category(usg_badge_color, assist_badge_color)
    catch_shoot_badge_color = determine_catch_shoot_badge_color(
        advanced_player_data['USG_PCT'],
        usg_percentiles,
        advanced_player_data['TS_PCT'],
        true_shooting_percentile
    )
    
    return facilitator_creator_badge_color, off_reb_badge_color, pace_badge_color, catch_shoot_badge_color, def_reb_badge_color, hustle_badge_color, shot_block_badge_color, steal_badge_color

# ALL Tab 2 callbacks
# update spider charts with 1st player selected from 1st dropdown
@app.callback(
    [Output('spider-offense', 'figure'),
     Output('spider-defense', 'figure')],
    [Input('dropdown1', 'value'),
     Input('dropdown2', 'value')]
)

def update_spider(selected_player_1, selected_player_2):

    # get player 1 information
    player1_info = pos_df.loc[pos_df['PLAYER_ID'] == selected_player_1]
    player1_info = player1_info.values.tolist()
    player1_info_data = list(chain.from_iterable(player1_info))
    player1_name = player1_info_data[1]

    # get player 2 information
    player2_info = pos_df.loc[pos_df['PLAYER_ID'] == selected_player_2]
    player2_info = player2_info.values.tolist()
    player2_info_data = list(chain.from_iterable(player2_info))
    player2_name = player2_info_data[1]

    # setting category names
    # off_categories = ['USG', 'AST', 'OREB', 'TS', 'PACE_PER40']
    off_categories = ['Usage %', 'Assists', 'Off. Rebound Rate', 'True Shooting %', 'Pace Factor/40 Minutes']
    def_categories = ['Def. Rebound Rate', 'Defense Efficiency', 'Blocks', 'Steals']

    # creating multiple trace offensive chart
    off_spider_figure = go.Figure()
    off_spider_figure.add_trace(go.Scatterpolar(r=player1_info_data[2:7], theta=off_categories, fill='toself', name=f'{player1_name}'))
    off_spider_figure.add_trace(go.Scatterpolar(r=player2_info_data[2:7], theta=off_categories, fill='toself', name=f'{player2_name}'))
    off_spider_figure.update_polars(radialaxis_range=(0,10))
    off_spider_figure.update_polars(radialaxis_dtick=1)

    # creating multiple trace defensive chart
    def_spider_figure = go.Figure()
    def_spider_figure.add_trace(go.Scatterpolar(r=player1_info_data[7:11], theta=def_categories, fill='toself', name=f'{player1_name}'))
    def_spider_figure.add_trace(go.Scatterpolar(r=player2_info_data[7:11], theta=def_categories, fill='toself', name=f'{player2_name}'))
    def_spider_figure.update_polars(radialaxis_range=(0,10))
    def_spider_figure.update_polars(radialaxis_dtick=1)

    return off_spider_figure, def_spider_figure 

# ALL tab 3 callbacks
# hover over data

# Run the app
if __name__ == '__main__':
    app.run_server(mode='external', debug=True)